In [2]:
from sqlalchemy import create_engine, text

dbpath = 'Chinook_Sqlite.sqlite'
connection_string = f"sqlite:///Chinook_Sqlite.sqlite"

engine = create_engine(connection_string)

with engine.connect() as connection:
    result = connection.execute(text("select * from album limit 3"))
    for row in result:
        print(row)

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)


In [3]:
table_details = {
    "customer": "stores customer’s data.",
    "employee": "stores all employee information as well as the organization structure such as who reports to whom.",
    "genre": "stores a list of media genres.",
    "invoice": "stores invoice data.",
    "invoice_item": "stores invoice line items.",
    "media_type": "stores media types such as MPEG audio file, ACC audio file, etc.",
    "artist": "stores a list of artists.",
    "album": "stores music albums.",
    "track": "stores songs by track name, length, and genre.",
    "playlist": "stores playlists data.",
    "playlist_track": "stores the relationship between playlists and tracks."
}

In [4]:
from llama_index.core import SQLDatabase
# tables = ["customer","artist"]
sql_database = SQLDatabase(engine, sample_rows_in_table_info=2)
sql_database

In [5]:
list(sql_database._all_tables)

['Album',
 'MediaType',
 'Customer',
 'PlaylistTrack',
 'Artist',
 'Invoice',
 'Genre',
 'Track',
 'Playlist',
 'Employee',
 'InvoiceLine']

In [6]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "<input key here>"
openai.api_key = os.environ["OPENAI_API_KEY"]

https://gpt-index.readthedocs.io/en/latest/examples/callbacks/TokenCountingHandler.html

In [7]:
import tiktoken
from llama_index.core.callbacks import CallbackManager, TokenCountingHandler
token_counter = TokenCountingHandler(
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo").encode
)

callback_manager = CallbackManager([token_counter])

In [8]:
from llama_index.core import ServiceContext, PromptHelper
from llama_index.llms.openai import OpenAI
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

service_context = ServiceContext.from_defaults(
  llm=llm,callback_manager=callback_manager
)

C:\Users\dkmsf\AppData\Local\Temp\ipykernel_14956\4046526135.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [9]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    service_context=service_context
)

In [24]:
query_str = "name top 3 most sold albums from the store"
# query_str = "can you provide information about costumer 22??"
response = query_engine.query(query_str)

In [25]:
print(response)

The top 3 most sold albums from the store are "Minha Historia" with 27 sales, "Greatest Hits" with 26 sales, and "Unplugged" with 25 sales.


In [12]:
print(token_counter.total_llm_token_count)

1012


In [13]:
token_counter.reset_counts()